In [188]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError
from datetime import date, datetime
from thefuzz import fuzz

pd.options.mode.chained_assignment = None  # default='warn'

from utils.configuration import settings
from utils.utils import *
from tft.api import *


c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344
PATCH = date(2022, 7, 1)

TARGETNAME = 'placement'

In [3]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [4]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,hnp_Mtq6VeuCan6mCb9lZ2drtbfD1FTcEwn0posRI9KMq48,Sb1w6C77m5sMRVQY70Ft0Wm6hP6sfcofnLr2h-chyBm9LNQ,xz-O4rBbMUaS5JRcbMNczdpbrZ7T3ioSO-ceKbC6DqNjMN...,Icey,4661,1656977400000,452,hnp_Mtq6VeuCan6mCb9lZ2drtbfD1FTcEwn0posRI9KMq48,Icey,228,I,53,33,False,False,True,True
1,asx_D8Lm0t2B1o1R3Z2BpSOfaQg_7QdJtFoBvrcxoo-q0-08,QPvzSqwQb-JfMZQtxZ70iWanvPAUD1kd5pa36rULx4kDCo...,bJuSIWIH7bFXcFJryZwWyaoywd4nAAaWQeNRKd0eIS9J6r...,Xellerex,3378,1657177208756,193,asx_D8Lm0t2B1o1R3Z2BpSOfaQg_7QdJtFoBvrcxoo-q0-08,Xellerex,405,I,204,182,False,False,True,False
2,TTdQLI20SMuncZY88NRHHgNCD5CHQgrhPj12GDATzQMxfBeu,qrsgEF7lO1d0QhWURVkdAVEurYh8nc9mdPJv_sUv8eckcx...,1AYfU2oGBkLPXyk_-EhTnN-85BEsUysY1uhjEMZvxaq3Oh...,uuFaded,5194,1657177425000,1,TTdQLI20SMuncZY88NRHHgNCD5CHQgrhPj12GDATzQMxfBeu,uuFaded,344,I,116,81,False,False,True,False
3,YsfIZVAbroGmQamvWM84NEhlz0qB6ahCz_OIHrXUfhPQdHI,XtPFF5eWcMNIGlXhRGEBWQOa7ITMO_xPt4FId9SLTfXnNQ,VTkKEB8vMbUdZIhVt8qlxsRdoTbG99kJ-EALAwzhCnBlB2...,SagecR,23,1657138589000,114,YsfIZVAbroGmQamvWM84NEhlz0qB6ahCz_OIHrXUfhPQdHI,SagecR,208,I,124,112,False,False,True,True
4,L8ZnO_dEDQYyVL3zf69oHA06mg20D9pfgz1jirkoQ0Fr3NT8,ow6PxI-kQfLEhVECJaHTpK3QXrzBJfuvx_bE5awGESzOBz...,ayhyBBmn8zZTMxBUHvb7IzY7B9sHpcTBYxmMX0TdYH_2jA...,yourrich,5169,1657166313000,230,L8ZnO_dEDQYyVL3zf69oHA06mg20D9pfgz1jirkoQ0Fr3NT8,yourrich,200,I,116,81,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,0EeWt-iA7o1vI6KFyo2c87Q217sLyUe-hP2sAtki_vV5X4w,k6EQ2Gp762ymBLMdGqoobYdJ6ERoi-GLXHOwRd_gmc8WIw,urayq63QvvrNY0FlZd--3hXu4kIh0m0suI3uto6JfUm6wS...,Yiggles,4277,1657167455000,125,0EeWt-iA7o1vI6KFyo2c87Q217sLyUe-hP2sAtki_vV5X4w,Yiggles,416,I,140,106,False,False,True,False
483,09EFThM5TgTXzif_RPaFVfsokwq2WL37K981PxyftyY8cK...,tHzAHp2REzHELqMpYTLV0xUR5QFlZRxHKhH1deem5fQEzp...,2h7R8kw9J0blIumXPKQX_eM_oQnVKpjPyRjjEFMYMhSsbw...,SewBerLow,29,1657027042000,1,09EFThM5TgTXzif_RPaFVfsokwq2WL37K981PxyftyY8cK...,SewBerLow,301,I,77,46,False,False,True,True
484,xdi366oQg7491x4QH2Mdqn_lpHIPhETKzmOJMrbuY1uaYvE,gzJZZ75BQD_7hCJbbnZCxFsi0-QrUrusiiz0KMgSP0O38A,kxWE95ML5C_TqrNyTlWp6zSl8DB6x3nOzzccczDdKDy265...,XTheFarmerX,28,1657154751216,195,xdi366oQg7491x4QH2Mdqn_lpHIPhETKzmOJMrbuY1uaYvE,XTheFarmerX,468,I,122,99,False,False,True,True
485,7JdDtGe92I5Gia2DeqExCZI9RbKJbhQ507oHOt3_tIyW1d65,OFytNbs-ZkQhryYFwJYexOxWEg2NW4i9jiXuLw9rbkJogG...,AB2wLDcC047q00ZidhkOvT2bmrzDgPy31MqZUWtt5SWozF...,navioot,3378,1657177374000,1,7JdDtGe92I5Gia2DeqExCZI9RbKJbhQ507oHOt3_tIyW1d65,navioot,255,I,164,140,False,False,True,False


In [5]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

2022-07-07 17:50:18,909 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_CN Ehtel .json.gz'
2022-07-07 17:50:19,350 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_Biscuitpiggy.json.gz'
2022-07-07 17:50:19,376 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_howdouplaylol.json.gz'
2022-07-07 17:50:19,424 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_Whae.json.gz'
2022-07-07 17:50:19,537 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_Smalding.json.gz'
2022-07-07 17:50:19,539 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_MÐB Ysy A.json.gz'
2022-07-07 17:50:19,657 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_KYTumbler.json.gz'
2022-07-07 17:50:19,658 [ERROR] [Errno 2] No such file or directory: 'assets\\matches_detail_na1_TaurusTard.json.gz'
2022-07-07 17:50:19,750 [ERROR] [Errno 2] No such file or directory: '

In [6]:
len(uniq_matches_id)

7533

In [7]:
len(matches_asset)

8880

In [8]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [9]:
len(uniq_matches)

7533

In [10]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [11]:
latest_matches = [match for match in uniq_matches if (
    LATEST_RELEASE in match['info']['game_version'])]


In [12]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [13]:
len(latest_matches)

6958

In [14]:
len(latest_patch_matches)

4094

In [15]:
# latest_matches[0]['info']

In [16]:
from typing import List


def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [17]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)

In [18]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4361396724',
 'placement': 6,
 'augment0': 'TFT6_Augment_Electrocharge2',
 'augment1': 'TFT6_Augment_ForceOfNature',
 'augment2': 'TFT6_Augment_ComponentGrabBag',
 'Set7_Assassin': 1,
 'Set7_Astral': 0,
 'Set7_Bruiser': 1,
 'Set7_Cavalier': 0,
 'Set7_Guardian': 0,
 'Set7_Guild': 1,
 'Set7_Legend': 0,
 'Set7_Ragewing': 1,
 'Set7_Swiftshot': 1,
 'Set7_Tempest': 1,
 'Set7_Warrior': 0,
 'Set7_Whispers': 0,
 'TFT7_Thresh': 2,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Thresh_item0': 'WarmogsArmor',
 'TFT7_Qiyana': 2,
 'TFT7_Shen': 2,
 'TFT7_Varus': 1,
 'TFT7_Varus_item0': 'UnstableConcoction',
 'TFT7_Hecarim': 2,
 'TFT7_Hecarim_item0': 'Redemption',
 'TFT7_Talon': 1,
 'TFT7_Talon_item0': 'ThiefsGloves',
 'TFT7_Talon_item1': 'InfinityEdge',
 'TFT7_Talon_item2': 'JeweledGauntlet',
 'TFT7_Xayah': 2,
 'TFT7_Xayah_item0': 'Quicksilver',
 'TFT7_Xayah_item1': 'GuinsoosRageblade',
 'TFT7_Xayah_item2': 'InfinityEdge',
 'TFT7_Ornn': 1,
 'TFT7_Or

In [19]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)

In [20]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Astral,Set7_Bruiser,Set7_Cavalier,Set7_Guardian,...,TFT7_TahmKench_item2,TFT7_Thresh_item1,TFT7_Yone_item2,TFT7_Thresh_item2,TFT7_Aatrox_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Sett_item2,TFT7_Heimerdinger_item2
0,NA1_4361396724,6,TFT6_Augment_Electrocharge2,TFT6_Augment_ForceOfNature,TFT6_Augment_ComponentGrabBag,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4361396724,8,TFT6_Augment_CelestialBlessing2,TFT6_Augment_BandOfThieves2,NaN,0.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4361396724,2,TFT6_Augment_CelestialBlessing2,TFT7_Augment_BruiserEmblem2,TFT7_Augment_BruiserTitanicStrength,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4361396724,7,TFT6_Augment_ThreesCompany,TFT6_Augment_RadiantRelics,TFT6_Augment_PortableForge,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4361396724,3,TFT6_Augment_SunfireBoard,TFT6_Augment_Electrocharge3,TFT6_Augment_PortableForge,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55659,NA1_4363985993,8,TFT6_Augment_MaxLevel10,TFT6_Augment_Diversify2,TFT7_Augment_BruiserTitanicStrength,0.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55660,NA1_4363985993,7,TFT7_Augment_GuildEmblem2,TFT7_Augment_AxiomArc2,TFT6_Augment_TargetDummies,0.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55661,NA1_4363985993,1,TFT6_Augment_ForceOfNature,TFT6_Augment_Ascension,TFT6_Augment_PortableForge,1.0,NaN,2.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55662,NA1_4363985993,6,TFT6_Augment_RadiantRelics,TFT7_Augment_CannoneerEmblem,TFT6_Augment_CyberneticShell2,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Astral,Set7_Bruiser,Set7_Cavalier,Set7_Guardian,...,TFT7_TahmKench_item2,TFT7_Thresh_item1,TFT7_Yone_item2,TFT7_Thresh_item2,TFT7_Aatrox_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Sett_item2,TFT7_Heimerdinger_item2
0,NA1_4361396724,6,TFT6_Augment_Electrocharge2,TFT6_Augment_ForceOfNature,TFT6_Augment_ComponentGrabBag,1.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4361396724,8,TFT6_Augment_CelestialBlessing2,TFT6_Augment_BandOfThieves2,NaN,0.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4361396724,2,TFT6_Augment_CelestialBlessing2,TFT7_Augment_BruiserEmblem2,TFT7_Augment_BruiserTitanicStrength,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4361396724,7,TFT6_Augment_ThreesCompany,TFT6_Augment_RadiantRelics,TFT6_Augment_PortableForge,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4361396724,3,TFT6_Augment_SunfireBoard,TFT6_Augment_Electrocharge3,TFT6_Augment_PortableForge,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55659,NA1_4363985993,8,TFT6_Augment_MaxLevel10,TFT6_Augment_Diversify2,TFT7_Augment_BruiserTitanicStrength,0.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55660,NA1_4363985993,7,TFT7_Augment_GuildEmblem2,TFT7_Augment_AxiomArc2,TFT6_Augment_TargetDummies,0.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55661,NA1_4363985993,1,TFT6_Augment_ForceOfNature,TFT6_Augment_Ascension,TFT6_Augment_PortableForge,1.0,NaN,2.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55662,NA1_4363985993,6,TFT6_Augment_RadiantRelics,TFT7_Augment_CannoneerEmblem,TFT6_Augment_CyberneticShell2,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [22]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [23]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)

# Output dataframes

In [24]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)

# Preprocessing

In [25]:
raw_df = matches_league_df.copy()
def impute(df):
    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(0)
    for name in df.select_dtypes("object"):
        df[name] = df[name].fillna("None")
    return df
raw_df = impute(raw_df)

In [43]:
match_id_df = raw_df['match_id']
X = raw_df.drop(['match_id'], axis=1)
y = X.pop(TARGETNAME)
X.fillna('', inplace=True)
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()
traits_col = [s for s in numeric_cols if "Set7" in s]
units_col = [s for s in numeric_cols if "TFT7" in s]
augments_col = ['augment0', 'augment1', 'augment2']
items_col = [s for s in categorical_cols if s not in augments_col]
df_unique = X.nunique().to_frame().reset_index()
df_unique.columns = ['Variable','DistinctCount']
unique_items_set = {y for col in items_col for y in X[col].unique().tolist()}
unique_augments_set = {y for col in augments_col for y in X[col].unique().tolist()}
X[f'items_count'] = X[items_col].apply(lambda row: sum(x != 'None' for x in row), axis=1)
X[f'traits_sum'] = X[traits_col].sum(axis=1)
X[f'units_sum'] = X[units_col].sum(axis=1)
X.iloc[X[f'units_sum'].idxmax()]

augment0          TFT7_Augment_MageConference
augment1              TFT6_Augment_ThriftShop
augment2             TFT6_Augment_SecondWind1
Set7_Assassin                             0.0
Set7_Astral                               1.0
                             ...             
TFT7_Zoe_item1                           None
TFT7_Zoe_item2                           None
items_count                                12
traits_sum                                9.0
units_sum                                30.0
Name: 21889, Length: 270, dtype: object

In [27]:
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()
X[numeric_cols] = X[numeric_cols].applymap(np.int64)
matches_df = X.copy()
matches_df[TARGETNAME] = y

In [85]:
def get_unit_items_ranking(df: matches_df, unit: str):
    # filter and melt the dataframe
    df = df.filter(regex=f'placement|{unit}_item0|{unit}_item1|{unit}_item2')
    df[f'unit'] = f'{unit}' # fill in current unit
    # join 3 items to 1 column
    df[f'{unit}_items'] = df[[f'{unit}_item0', f'{unit}_item1', f'{unit}_item2']].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
    # sort items for unique combination
    df[f'{unit}_items'] = df[f'{unit}_items'].apply(lambda x: ', '.join(sorted(x.split(', '))))
    df = df.filter(regex=f'placement|{unit}_items|unit')
    m = df.melt(
        ['placement',f'unit'], value_name=f'{unit}_items_grp') #, value_vars=[f'{unit}_items', f'{unit}']
    # group and aggregate mean/median average_placement
    dct = {'value_count': (f'{unit}_items_grp', 'count'),
           'average_placement': ('placement', 'mean')}
    return m.groupby([f'unit', f'{unit}_items_grp'], as_index=False).agg(**dct).sort_values(by='average_placement')

In [86]:
get_unit_items_ranking(df = matches_df[:2], unit='TFT7_Zoe')

,unit,TFT7_Zoe_items_grp,value_count,average_placement
0,TFT7_Zoe,"None, None, None",2,7.0


In [87]:
# Get top5 
top5_items_list = []
for unit in units_col:
    df = get_unit_items_ranking(df = matches_df, unit=unit)
    df = df[df['value_count']>=12][:5] #Top 5 with counts >= 12
    top5_items_list.extend(df.values)

In [88]:
top5_items_list = pd.DataFrame(top5_items_list, columns=['unit', 'items',	'value_count',	'average_placement'])

In [31]:
top5_items_list.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_top5_items.csv'), index=False)

In [32]:
pd.options.display.max_colwidth = 100

In [89]:
top5_items_list.groupby('unit').head(1)

,unit,items,value_count,average_placement
0,TFT7_Aatrox,"None, None, Zephyr",13,2.230769
5,TFT7_Anivia,"ArchangelsStaff, GuinsoosRageblade, Morellonomicon",18,2.666667
10,TFT7_AoShin,"ArchangelsStaff, SpearOfShojin, SpearOfShojin",12,3.000000
15,TFT7_Ashe,"StatikkShiv, ZekesHerald, ZekesHerald",14,2.857143
20,TFT7_AurelionSol,"ArchangelsStaff, HextechGunblade, MageEmblemItem",31,4.064516
24,TFT7_Bard,"None, None, ThiefsGloves",13,1.153846
29,TFT7_Braum,"None, None, TitansResolve",14,3.500000
34,TFT7_Corki,"Deathblade, GuinsoosRageblade, StatikkShiv",13,3.153846
39,TFT7_Diana,"FrozenHeart, HextechGunblade, RedBuff",14,2.857143
44,TFT7_DragonBlue,"Quicksilver, RevelEmblemItem, StatikkShiv",21,2.714286


In [189]:
def get_unit_comp_ranking(df: matches_df):
    # filter and melt the dataframe
    df = df.filter(['placement']+units_col)
    # join units to 1 column
    df['comp'] = df[units_col].apply(lambda row: ', '.join(row[row > 0].index.values.astype(str)), axis=1)
    df['comp'] = df['comp'].str.replace('TFT7_','') #.split('_',1).str[-1]
    df = df.filter(['placement', 'comp'])
    m = df.melt(
        ['placement'], value_name=f'comp_grp')
    # group and aggregate mean/median average_placement
    dct = {'value_count': (f'comp_grp', 'count'),
           'average_placement': ('placement', 'mean')}
    return m.groupby([f'comp_grp'], as_index=False).agg(**dct).sort_values(by='average_placement')

In [141]:
# get_unit_comp_ranking(df = matches_df[:50])

In [270]:
# Get top5 
comp_df = get_unit_comp_ranking(df = matches_df[:20])

In [271]:
top5_comp_list = []
m = comp_df[comp_df['value_count']>=1] #[:5] #Top 5 with counts >= 12
top5_comp_list.extend(m.values)
comp_ranking_df = pd.DataFrame(top5_comp_list, columns=['comp', 'value_count', 'average_placement'])

In [272]:
comp_ranking_df

,comp,value_count,average_placement
0,"Anivia, Corki, DragonGold, Jinx, Lulu, Sona, TrainerDragon, Tristana",1,1.0
1,"Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah, Yasuo, Zoe",1,1.0
2,"Bard, DragonPurple, Ornn, Ryze, Shen, Sylas, Zoe",1,2.0
3,"Anivia, Ashe, DragonGreen, Gnar, Lulu, Neeko, Soraka, Yasuo",1,2.0
4,"Bard, Corki, Gnar, Neeko, Ornn, Yasuo, Zoe",1,2.0
5,"Anivia, Bard, DragonGreen, Gnar, Lulu, Neeko, Soraka",1,3.0
6,"Bard, DragonGold, DragonGreen, Shyvana, Swain, Yasuo",1,3.0
7,"Ezreal, Illaoi, Ornn, Shen, Skarner, Twitch, Varus, Xayah",1,4.0
8,"DragonPurple, Ornn, Pyke, Qiyana, Shen, Sylas, Thresh",1,4.0
9,"Ashe, Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch",1,4.0


In [236]:
comp_ranking_df #.groupby('comp').head(1)

,comp,value_count,average_placement
0,"Bard, DragonBlue, Hecarim, Nunu, Ornn, Qiyana, Shen, Talon, Yasuo",1,1.0
1,"Bard, Hecarim, Pyke, Shyvana, Soraka, Swain, Talon, Twitch, Xayah",1,1.0
2,"Anivia, Bard, DragonGreen, Gnar, Neeko, Pyke, Talon, Yasuo",1,1.0
3,"Bard, DragonPurple, Heimerdinger, Lulu, Sylas, TrainerDragon, Tristana, Zoe",1,1.0
4,"DragonPurple, Elise, Ornn, Pyke, Sylas, Talon, Thresh",1,1.0
...,...,...,...
425,"DragonPurple, Olaf, Ornn, Shen, Sylas, Yasuo",1,8.0
426,"DragonPurple, Olaf, Ornn, Qiyana, Shen, Sylas, Talon",1,8.0
427,"Bard, Hecarim, Ornn, Qiyana, Sejuani, Shen, Twitch, Xayah",1,8.0
428,"Ezreal, Lillia, Qiyana, Sejuani, Talon, Twitch, Varus",1,8.0


In [238]:
from statistics import mode
import jellyfish

import pandas as pd

df = pd.DataFrame({'Code': ['abc', 'abc', 'abc', 'abcc', 'abcc', 'zxc'],
                   'Description': ['ABC String', 'abc string', 'ABC String and sth', 'abc sth else', 'zxc sth else', 'zxc zxc'],
                #    'Value': [10, 20, 30, 40, 100]
                   })

df_list = []
for grp,df in df.groupby('Code'):
    df['distance'] = df['Description'].apply(lambda x : fuzz.token_set_ratio(x, mode(df['Description'])))
    # df['Description'] =  mode(df['Description'])
    df_list.append(df[df['distance'] > 10])

df = pd.concat(df_list)

In [239]:
df

,Code,Description,distance
0,abc,ABC String,100
1,abc,abc string,100
2,abc,ABC String and sth,100
3,abcc,abc sth else,100
4,abcc,zxc sth else,83
5,zxc,zxc zxc,100


In [275]:
','.join(units_col)

'TFT7_Aatrox,TFT7_Anivia,TFT7_AoShin,TFT7_Ashe,TFT7_AurelionSol,TFT7_Bard,TFT7_Braum,TFT7_Corki,TFT7_Diana,TFT7_DragonBlue,TFT7_DragonGold,TFT7_DragonGreen,TFT7_DragonPurple,TFT7_Elise,TFT7_Ezreal,TFT7_Gnar,TFT7_Hecarim,TFT7_Heimerdinger,TFT7_Illaoi,TFT7_Jinx,TFT7_Karma,TFT7_Kayn,TFT7_LeeSin,TFT7_Leona,TFT7_Lillia,TFT7_Lulu,TFT7_Nami,TFT7_Neeko,TFT7_Nidalee,TFT7_Nunu,TFT7_Olaf,TFT7_Ornn,TFT7_Pyke,TFT7_Qiyana,TFT7_Ryze,TFT7_Sejuani,TFT7_Senna,TFT7_Sett,TFT7_Shen,TFT7_Shyvana,TFT7_Skarner,TFT7_Sona,TFT7_Soraka,TFT7_Swain,TFT7_Sylas,TFT7_TahmKench,TFT7_Talon,TFT7_Taric,TFT7_Thresh,TFT7_TrainerDragon,TFT7_Tristana,TFT7_Twitch,TFT7_Varus,TFT7_Vladimir,TFT7_Volibear,TFT7_Xayah,TFT7_Yasuo,TFT7_Yone,TFT7_Zoe'

In [273]:
comp_ranking_df['comp'][:-1].values[-1]

'Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah'

In [276]:
comp_ranking_df.groupby('comp')['comp'].apply(lambda x : fuzz.token_set_ratio(x, ','.join(units_col)))

comp
Anivia, Ashe, Braum, Corki, Talon, Taric                                      6
Anivia, Ashe, DragonGreen, Gnar, Lulu, Neeko, Soraka, Yasuo                   3
Anivia, Bard, DragonGreen, Gnar, Lulu, Neeko, Soraka                          4
Anivia, Corki, DragonGold, Jinx, Lulu, Sona, TrainerDragon, Tristana          5
Anivia, DragonGreen, Gnar, Lulu, Neeko, Talon, Twitch                         3
Ashe, DragonGreen, Karma, LeeSin, Leona, Neeko, Taric                         4
Ashe, Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch                     3
Bard, Corki, Gnar, Neeko, Ornn, Yasuo, Zoe                                    5
Bard, DragonGold, DragonGreen, Shyvana, Swain, Yasuo                          2
Bard, DragonPurple, Ornn, Ryze, Shen, Sylas, Zoe                              6
Braum, DragonGreen, Gnar, Neeko, Sejuani, Taric                               3
Corki, DragonBlue, Lulu, Ornn, Sona, TahmKench, TrainerDragon, Tristana       8
Corki, DragonGold, Jinx, Lulu, Sona

In [255]:
df_list = []
for grp,df in comp_ranking_df.groupby('comp'):
    df['distance'] = df['comp'].apply(lambda x : fuzz.token_set_ratio(x, 'Aatrox, DragonGold, Kayn, Shen, Twitch, Xayah, Zoe'))
    # df['Description'] =  mode(df['Description'])
    df_list.append(df[df['distance'] > 10])

df = pd.concat(df_list)

In [256]:
df

,comp,value_count,average_placement,distance
429,"Aatrox, DragonGold, Kayn, Shen, Twitch, Xayah, Zoe",1,8.0,100
249,"Aatrox, DragonGold, Kayn, Shen, Volibear, Xayah, Zoe",1,5.0,91
318,"Aatrox, DragonGold, Kayn, Talon, Volibear, Yasuo, Zoe",1,6.0,74
198,"Anivia, Ashe, Bard, DragonGreen, Gnar, Lulu, Neeko",1,4.0,39
41,"Anivia, Ashe, Bard, DragonGreen, Gnar, Lulu, Neeko, Soraka",1,1.0,36
...,...,...,...,...
247,"Illaoi, Olaf, Ornn, Sylas, Talon, Twitch, Varus",1,5.0,45
248,"Illaoi, Ornn, Shen, Sylas, Varus, Xayah",1,5.0,45
381,"Jinx, Lulu, Shen, Sona, TahmKench, TrainerDragon, Tristana",1,8.0,40
43,"Neeko, Pyke, Shen, Shyvana, Talon, Twitch, Xayah, Yasuo",1,1.0,57


In [280]:
"""
This is a more complex example on performing clustering on large scale dataset.
This examples find in a large set of sentences local communities, i.e., groups of sentences that are highly
similar. You can freely configure the threshold what is considered as similar. A high threshold will
only find extremely similar sentences, a lower threshold will find more sentence that are less similar.
A second parameter is 'min_community_size': Only communities with at least a certain number of sentences will be returned.
The method for finding the communities is extremely fast, for clustering 50k sentences it requires only 5 seconds (plus embedding comuptation).
In this example, we download a large set of questions from Quora and then find similar questions in this set.
"""
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time


# Model for computing sentence embeddings. We use one trained for similar questions detection
model = SentenceTransformer('all-MiniLM-L6-v2')

# We donwload the Quora Duplicate Questions Dataset (https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs)
# and find similar question in it
url = "http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
dataset_path = "quora_duplicate_questions.tsv"
max_corpus_size = 50000  # We limit our corpus to only the first 50k questions


# Check if the dataset exists. If not, download and extract
# Download dataset if needed
if not os.path.exists(dataset_path):
    print("Download dataset")
    util.http_get(url, dataset_path)

# Get all unique sentences from the file
corpus_sentences = set()
with open(dataset_path, encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
    for row in reader:
        corpus_sentences.add(row['question1'])
        corpus_sentences.add(row['question2'])
        if len(corpus_sentences) >= max_corpus_size:
            break

corpus_sentences = list(corpus_sentences)
print("Encode the corpus. This might take a while")
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)


print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.75)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus_sentences[sentence_id])


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\furyx\miniconda3\envs\tft\lib\logging\__init__.py", line 1104, in emit
    self.flush()
  File "c:\Users\furyx\miniconda3\envs\tft\lib\logging\__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 22] Invalid argument
Call stack:
  File "c:\Users\furyx\miniconda3\envs\tft\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\furyx\miniconda3\envs\tft\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
    app.start()
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "c:\Users\furyx\m

Encode the corpus. This might take a while


Batches: 100%|██████████| 782/782 [04:27<00:00,  2.92it/s]


Start clustering
Clustering done after 15.48 sec

Cluster 1, #103 Elements 
	 How can I improve my English grammar?
	 How can l improve my English？?
	 How do I Speak English fluently and build vocabulary?
	 ...
	 What is the best way to talk English fluently?
	 How can I take my spoken English to a higher level when I speak good enough but not fluently?
	 What are some ways to improve English?

Cluster 2, #86 Elements 
	 What is the easiest way to earn money from online?
	 How can one earn money online without investment?
	 What are some good ways to earn money online?
	 ...
	 How can I earn money online?
	 How do I legitimately earn online?
	 How should I make money online for free?

Cluster 3, #82 Elements 
	 How will the 500 & 1000 rupee note ban affect India?
	 How can the ban of 500 and 1000 rupee notes increase the Indian economy?
	 What will be the effects of demonetisation of 500 and 1000 notes on the Indian economy?
	 ...
	 What will be the effect of banning 500 and 1000 notes

In [282]:
corpus_embeddings

tensor([[-0.0078, -0.0859, -0.0018,  ..., -0.1256, -0.0099,  0.0327],
        [ 0.0147,  0.0226,  0.0047,  ..., -0.1259, -0.0681,  0.0254],
        [ 0.0130, -0.0286, -0.0135,  ..., -0.0297,  0.0214, -0.0957],
        ...,
        [-0.0774, -0.0424,  0.0089,  ...,  0.0348, -0.0015,  0.0014],
        [-0.0579,  0.0852, -0.1364,  ...,  0.0394, -0.0479, -0.0366],
        [ 0.0978, -0.0569, -0.0129,  ..., -0.0152,  0.0238, -0.0584]])

In [285]:
list(comp_ranking_df['comp'])

['Anivia, Corki, DragonGold, Jinx, Lulu, Sona, TrainerDragon, Tristana',
 'Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah, Yasuo, Zoe',
 'Bard, DragonPurple, Ornn, Ryze, Shen, Sylas, Zoe',
 'Anivia, Ashe, DragonGreen, Gnar, Lulu, Neeko, Soraka, Yasuo',
 'Bard, Corki, Gnar, Neeko, Ornn, Yasuo, Zoe',
 'Anivia, Bard, DragonGreen, Gnar, Lulu, Neeko, Soraka',
 'Bard, DragonGold, DragonGreen, Shyvana, Swain, Yasuo',
 'Ezreal, Illaoi, Ornn, Shen, Skarner, Twitch, Varus, Xayah',
 'DragonPurple, Ornn, Pyke, Qiyana, Shen, Sylas, Thresh',
 'Ashe, Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch',
 'DragonPurple, Ornn, Pyke, Qiyana, Shen, Sylas, Talon, Thresh',
 'Corki, DragonBlue, Lulu, Ornn, Sona, TahmKench, TrainerDragon, Tristana',
 'Hecarim, Ornn, Qiyana, Shen, Talon, Thresh, Varus, Xayah',
 'Ashe, DragonGreen, Karma, LeeSin, Leona, Neeko, Taric',
 'Braum, DragonGreen, Gnar, Neeko, Sejuani, Taric',
 'Anivia, DragonGreen, Gnar, Lulu, Neeko, Talon, Twitch',
 'Corki, DragonGold, J

In [286]:
corpus_sentences = list(comp_ranking_df['comp'])

In [287]:
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.90it/s]


In [302]:
#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=2, threshold=0.88)

In [303]:
#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus_sentences[sentence_id])


Cluster 1, #4 Elements 
	 Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah, Yasuo, Zoe
	 Ashe, Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch
	 Hecarim, Ornn, Qiyana, Shen, Talon, Thresh, Varus, Xayah
	 ...
	 Ashe, Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch
	 Hecarim, Ornn, Qiyana, Shen, Talon, Thresh, Varus, Xayah
	 Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah

Cluster 2, #3 Elements 
	 Anivia, Ashe, DragonGreen, Gnar, Lulu, Neeko, Soraka, Yasuo
	 Anivia, Bard, DragonGreen, Gnar, Lulu, Neeko, Soraka
	 Anivia, DragonGreen, Gnar, Lulu, Neeko, Talon, Twitch
	 ...
	 Anivia, Ashe, DragonGreen, Gnar, Lulu, Neeko, Soraka, Yasuo
	 Anivia, Bard, DragonGreen, Gnar, Lulu, Neeko, Soraka
	 Anivia, DragonGreen, Gnar, Lulu, Neeko, Talon, Twitch

Cluster 3, #2 Elements 
	 Anivia, Corki, DragonGold, Jinx, Lulu, Sona, TrainerDragon, Tristana
	 Corki, DragonGold, Jinx, Lulu, Sona, Thresh, TrainerDragon, Tristana
	 ...
	 Anivia, Corki, DragonGold, Jinx, Lulu, Sona, 

In [362]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering, DBSCAN
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Corpus with example sentences
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'A man is eating pasta.',
          'The girl is carrying a baby.',
          'The baby is carried by the woman',
          'A man is riding a horse.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.',
          'A cheetah is running behind its prey.',
          'A cheetah chases prey on across a field.'
          ]
corpus_embeddings = embedder.encode(corpus)

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\furyx\miniconda3\envs\tft\lib\logging\__init__.py", line 1104, in emit
    self.flush()
  File "c:\Users\furyx\miniconda3\envs\tft\lib\logging\__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 22] Invalid argument
Call stack:
  File "c:\Users\furyx\miniconda3\envs\tft\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\furyx\miniconda3\envs\tft\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
    app.start()
  File "c:\Users\furyx\miniconda3\envs\tft\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "c:\Users\furyx\m

Cluster  1
['A man is eating food.', 'A man is eating a piece of bread.', 'A man is eating pasta.']

Cluster  5
['The girl is carrying a baby.', 'The baby is carried by the woman']

Cluster  2
['A man is riding a horse.', 'A man is riding a white horse on an enclosed ground.']

Cluster  3
['A monkey is playing drums.', 'Someone in a gorilla costume is playing a set of drums.']

Cluster  4
['A cheetah is running behind its prey.', 'A cheetah chases prey on across a field.']



In [366]:
corpus = list(comp_ranking_df['comp'])

In [388]:
corpus_embeddings = embedder.encode(corpus)
# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)
# Perform kmean clustering
# clustering_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='single', distance_threshold=0.09) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model = DBSCAN(eps=0.5, min_samples=1)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.11it/s]


In [389]:
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

In [390]:
for i, cluster in clustered_sentences.items():
    print("Cluster ", i)
    print(cluster)
    print("")

Cluster  0
['Anivia, Corki, DragonGold, Jinx, Lulu, Sona, TrainerDragon, Tristana', 'Corki, DragonBlue, Lulu, Ornn, Sona, TahmKench, TrainerDragon, Tristana', 'Corki, DragonGold, Jinx, Lulu, Sona, Thresh, TrainerDragon, Tristana']

Cluster  1
['Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah, Yasuo, Zoe', 'Ashe, Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch', 'Hecarim, Ornn, Qiyana, Shen, Talon, Thresh, Varus, Xayah', 'Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah']

Cluster  2
['Bard, DragonPurple, Ornn, Ryze, Shen, Sylas, Zoe']

Cluster  3
['Anivia, Ashe, DragonGreen, Gnar, Lulu, Neeko, Soraka, Yasuo', 'Anivia, Bard, DragonGreen, Gnar, Lulu, Neeko, Soraka', 'Anivia, DragonGreen, Gnar, Lulu, Neeko, Talon, Twitch']

Cluster  4
['Bard, Corki, Gnar, Neeko, Ornn, Yasuo, Zoe']

Cluster  5
['Bard, DragonGold, DragonGreen, Shyvana, Swain, Yasuo']

Cluster  6
['Ezreal, Illaoi, Ornn, Shen, Skarner, Twitch, Varus, Xayah']

Cluster  7
['DragonPurple, Ornn, Pyke, Qiy

In [335]:
corpus[1]

'Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah, Yasuo, Zoe'

In [391]:
cluster_assignment

array([ 0,  1,  2,  3,  4,  3,  5,  6,  7,  1,  7,  0,  1,  8,  9,  3,  0,
       10,  1, 11], dtype=int64)

In [394]:
corpus = list(comp_ranking_df['comp'])
corpus_embeddings = embedder.encode(corpus)
clustering_model = DBSCAN(eps=0.5, min_samples=1)
predict=clustering_model.fit_predict(corpus_embeddings)

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.29it/s]


In [395]:
comp_ranking_df['group'] = pd.Series(predict, index=comp_ranking_df.index)

In [396]:
comp_ranking_df

,comp,value_count,average_placement,group
0,"Anivia, Corki, DragonGold, Jinx, Lulu, Sona, TrainerDragon, Tristana",1,1.0,0
1,"Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch, Xayah, Yasuo, Zoe",1,1.0,1
2,"Bard, DragonPurple, Ornn, Ryze, Shen, Sylas, Zoe",1,2.0,2
3,"Anivia, Ashe, DragonGreen, Gnar, Lulu, Neeko, Soraka, Yasuo",1,2.0,3
4,"Bard, Corki, Gnar, Neeko, Ornn, Yasuo, Zoe",1,2.0,4
5,"Anivia, Bard, DragonGreen, Gnar, Lulu, Neeko, Soraka",1,3.0,3
6,"Bard, DragonGold, DragonGreen, Shyvana, Swain, Yasuo",1,3.0,5
7,"Ezreal, Illaoi, Ornn, Shen, Skarner, Twitch, Varus, Xayah",1,4.0,6
8,"DragonPurple, Ornn, Pyke, Qiyana, Shen, Sylas, Thresh",1,4.0,7
9,"Ashe, Hecarim, Ornn, Qiyana, Sejuani, Shen, Talon, Twitch",1,4.0,1
